# Data Wrangling

In [8]:
# importing required libraries
import random
import datetime
import numpy as np
import pandas as pd
from random import randrange
from sklearn import preprocessing

pd.options.mode.chained_assignment = None

In [25]:
def _random_date(start,date_count):
    current = start
    while date_count > 0:
        curr = current + datetime.timedelta(days=randrange(42))
        yield curr
        date_count-=1
        
def generate_sample_data(row_count=100):
    
    startDate = datetime.datetime(2016, 1, 1,13)
    serial_number_sentinel = 1000
    user_id_sentinel = 5001
    product_id_sentinel = 101
    price_sentinel = 2000
    
    data_dict = {
    'Serial No': np.arange(row_count)+serial_number_sentinel,
    'Date': np.random.permutation(pd.to_datetime([x.strftime("%d-%m-%Y") 
                                                    for x in _random_date(startDate,
                                                                          row_count)]).date
                                  ),
    'User ID': np.random.permutation(np.random.randint(0,
                                                       row_count,
                                                       size=int(row_count/10)) + user_id_sentinel).tolist()*10,
    'Product ID': np.random.permutation(np.random.randint(0,
                                                          row_count,
                                                          size=int(row_count/10))+ product_id_sentinel).tolist()*10 ,
    'Quantity Purchased': np.random.permutation(np.random.randint(1,
                                                                  42,
                                                                  size=row_count)),
    'Price': np.round(np.abs(np.random.randn(row_count)+1)*price_sentinel,
                      decimals=2),
    'User Type':np.random.permutation([chr(random.randrange(97, 97 + 3 + 1)) 
                                            for i in range(row_count)])
    }
    
    for index in range(int(np.sqrt(row_count))): 
        data_dict['Price'][np.argmax(data_dict['Price'] == random.choice(data_dict['Price']))] = np.nan
        data_dict['User Type'][np.argmax(data_dict['User Type'] == random.choice(data_dict['User Type']))] = np.nan
        data_dict['Date'][np.argmax(data_dict['Date'] == random.choice(data_dict['Date']))] = np.nan
        data_dict['Product ID'][np.argmax(data_dict['Product ID'] == random.choice(data_dict['Product ID']))] = 0
        data_dict['Serial No'][np.argmax(data_dict['Serial No'] == random.choice(data_dict['Serial No']))] = -1
        data_dict['User ID'][np.argmax(data_dict['User ID'] == random.choice(data_dict['User ID']))] = -101
    
    df = pd.DataFrame(data_dict)
    
    return df

def describe_dataframe(df=pd.DataFrame()):
    
    print("\n\n")
    print("*"*30)
    print("About the Data")
    print("*"*30)
    
    print("Number of rows::",df.shape[0])
    print("Number of columns::",df.shape[1])
    print("\n")
    
    print("Column Names::",df.columns.values.tolist())
    print("\n")
    
    print("Column Data Types::\n",df.dtypes)
    print("\n")
    
    print("Columns with Missing Values::",df.columns[df.isnull().any()].tolist())
    print("\n")
    
    print("Number of rows with Missing Values::",len(pd.isnull(df).any(1).nonzero()[0].tolist()))
    print("\n")
    
    print("Sample Indices with missing data::",pd.isnull(df).any(1).nonzero()[0].tolist()[0:5])
    print("\n")
    
    print("General Stats::")
    print(df.info())
    print("\n")
    
    print("Summary Stats::")
    print(df.describe())
    print("\n")
    
    print("Dataframe Sample Rows::")
    display(df.head(5))
    
def cleanup_column_names(df,rename_dict={},do_inplace=True):
    if not rename_dict:
        return df.rename(columns={col: col.lower().replace(' ','_') 
                    for col in df.columns.values.tolist()}, 
                  inplace=do_inplace)
    else:
        return df.rename(columns=rename_dict,inplace=do_inplace)

def expand_user_type(u_type):
    if u_type in ['a','b']:
        return 'new'
    elif u_type == 'c':
        return 'existing'
    elif u_type == 'd':
        return 'loyal_existing'
    else:
        return 'error'

## Generate Sample Dataset

In [26]:
df = generate_sample_data(row_count=1000)

### Describe the Dataset

In [27]:
describe_dataframe(df)




******************************
About the Data
******************************
Number of rows:: 1000
Number of columns:: 7


Column Names:: ['Serial No', 'Date', 'User ID', 'Product ID', 'Quantity Purchased', 'Price', 'User Type']


Column Data Types::
 Serial No               int32
Date                   object
User ID                 int64
Product ID              int64
Quantity Purchased      int32
Price                 float64
User Type              object
dtype: object


Columns with Missing Values:: ['Date', 'Price']


Number of rows with Missing Values:: 59


Sample Indices with missing data:: [0, 1, 3, 7, 10]


General Stats::
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
Serial No             1000 non-null int32
Date                  969 non-null object
User ID               1000 non-null int64
Product ID            1000 non-null int64
Quantity Purchased    1000 non-null int32
Price                 970 non-null float64

C:\Users\Asghar\Anaconda3\lib\site-packages\ipykernel_launcher.py:69: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
C:\Users\Asghar\Anaconda3\lib\site-packages\ipykernel_launcher.py:72: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead


         Serial No      User ID   Product ID  Quantity Purchased        Price
count  1000.000000  1000.000000  1000.000000         1000.000000   970.000000
mean   1452.333000  5509.457000   584.168000           21.139000  2336.995464
std     386.014636   325.094045   292.509976           11.792911  1612.440087
min      -1.000000  -101.000000     0.000000            1.000000     8.960000
25%    1224.750000  5276.500000   314.000000           11.000000  1073.470000
50%    1480.500000  5529.000000   580.000000           22.000000  2035.710000
75%    1742.250000  5738.750000   850.000000           31.000000  3323.507500
max    1999.000000  5998.000000  1097.000000           41.000000  8960.800000


Dataframe Sample Rows::


,Serial No,Date,User ID,Product ID,Quantity Purchased,Price,User Type
0,1000,NaN,-101,0,28,NaN,n
1,1001,NaN,5395,411,25,4177.95,n
2,1002,2016-06-01,5439,488,17,2817.62,n
3,1003,NaN,5821,202,35,8576.58,n
4,1004,2016-01-25,5001,580,30,2174.16,n


### Rename Columns

In [28]:
print("Dataframe columns:\n{}".format(df.columns.tolist()))

Dataframe columns:
['Serial No', 'Date', 'User ID', 'Product ID', 'Quantity Purchased', 'Price', 'User Type']


In [29]:
cleanup_column_names(df)

In [30]:
print("Dataframe columns:\n{}".format(df.columns.tolist()))

Dataframe columns:
['serial_no', 'date', 'user_id', 'product_id', 'quantity_purchased', 'price', 'user_type']


### Sort Rows on defined attributes

In [31]:
display(df.sort_values(['serial_no', 'price'], 
                         ascending=[True, False]).head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
815,-1,2016-08-01,5675,570,27,8960.80,b
243,-1,2016-10-02,5565,806,10,6287.44,b
640,-1,2016-09-01,5834,729,18,5350.90,d
966,-1,2016-01-31,5377,895,32,5113.80,b
62,-1,2016-09-02,5792,864,22,4615.80,b


### Rearrange Columns in a Dataframe

In [32]:
display(df[['serial_no','date','user_id','user_type',
              'product_id','quantity_purchased','price']].head())

,serial_no,date,user_id,user_type,product_id,quantity_purchased,price
0,1000,NaN,-101,n,0,28,NaN
1,1001,NaN,5395,n,411,25,4177.95
2,1002,2016-06-01,5439,n,488,17,2817.62
3,1003,NaN,5821,n,202,35,8576.58
4,1004,2016-01-25,5001,n,580,30,2174.16


### Filtering Columns

Using Column Index

In [33]:
# print 10 values from column at index 3
print(df.iloc[:,3].values[0:10])

[  0 411 488 202 580 601 597 475 212 846]


Using Column Name

In [35]:
# print 10 values of quantity purchased
print(df.quantity_purchased.values[0:10])

[28 25 17 35 30 31 36 11 21  5]


Using Column Data Type

In [36]:
# print 10 values of columns with data type float
print(df.select_dtypes(include=['float64']).values[:10,0])

[    nan 4177.95 2817.62 8576.58 2174.16 1571.84 2036.79 1962.66 3901.22
 2929.12]


### Filtering Rows

Select Specific Rows

In [37]:
display(df.iloc[[10,501,20]])

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
10,1010,NaN,5637,850,3,2132.77,n
501,1501,2016-03-02,5395,411,4,1856.70,a
20,1020,2016-01-13,5500,619,7,924.49,n


Exclude Specific Row Indices

In [38]:
display(df.drop([0,24,51], axis=0).head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
1,1001,NaN,5395,411,25,4177.95,n
2,1002,2016-06-01,5439,488,17,2817.62,n
3,1003,NaN,5821,202,35,8576.58,n
4,1004,2016-01-25,5001,580,30,2174.16,n
5,1005,2016-01-25,5405,601,31,1571.84,n


Conditional Filtering

In [40]:
display(df[df.quantity_purchased>25].head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
0,1000,NaN,-101,0,28,NaN,n
3,1003,NaN,5821,202,35,8576.58,n
4,1004,2016-01-25,5001,580,30,2174.16,n
5,1005,2016-01-25,5405,601,31,1571.84,n
6,1006,2016-11-02,5392,597,36,2036.79,n


Offset from top of Dataframe

In [41]:
display(df[100:].head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
100,1100,2016-02-01,5192,1072,12,5509.98,c
101,1101,2016-01-26,5395,411,24,1005.79,b
102,1102,2016-01-27,5439,488,41,3158.98,b
103,-1,2016-03-02,5821,202,11,2769.42,a
104,1104,2016-11-02,5001,580,24,1677.87,b


Offset from bottom of Dataframe

In [42]:
display(df[-10:].head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
990,1990,2016-01-30,5998,324,8,4092.65,a
991,1991,2016-06-01,5122,795,31,4149.18,a
992,1992,2016-02-01,5069,821,37,4581.86,d
993,1993,2016-01-18,5997,231,19,2596.50,b
994,1994,2016-01-29,5429,698,6,2683.09,d


### Data Type Conversion

In [44]:
df['date'] = pd.to_datetime(df.date)
# compare dtypes of the original df with this one
print(df.dtypes)

serial_no                      int32
date                  datetime64[ns]
user_id                        int64
product_id                     int64
quantity_purchased             int32
price                        float64
user_type                     object
dtype: object


### Apply/Map Usage

In [46]:
df['user_class'] = df['user_type'].map(expand_user_type)
display(df.tail())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class
995,1995,2016-06-02,5229,378,38,2962.47,c,existing
996,1996,2016-01-13,5639,1002,26,4674.63,b,new
997,1997,2016-10-01,5787,180,25,4060.43,a,new
998,1998,2016-10-02,5463,868,8,3867.00,b,new
999,1999,2016-06-01,5283,918,27,2370.00,b,new


Using Apply to get attribute ranges

In [47]:
display(df.select_dtypes(include=[np.number]).apply(lambda x: 
                                                        x.max()- x.min()))

serial_no             2000.00
user_id               6099.00
product_id            1097.00
quantity_purchased      40.00
price                 8951.84
dtype: float64

Apply Week: Extract data from week

In [48]:
df['purchase_week'] = df[['date']].applymap(lambda dt:dt.week 
                                                if not pd.isnull(dt.week) 
                                                else 0)

In [49]:
display(df.head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week
0,1000,NaT,-101,0,28,NaN,n,error,0
1,1001,NaT,5395,411,25,4177.95,n,error,0
2,1002,2016-06-01,5439,488,17,2817.62,n,error,22
3,1003,NaT,5821,202,35,8576.58,n,error,0
4,1004,2016-01-25,5001,580,30,2174.16,n,error,4


### Missing Values

Drop Rows with Misssing Dates

In [51]:
df_dropped = df.dropna(subset=['date'])
display(df_dropped.head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week
2,1002,2016-06-01,5439,488,17,2817.62,n,error,22
4,1004,2016-01-25,5001,580,30,2174.16,n,error,4
5,1005,2016-01-25,5405,601,31,1571.84,n,error,4
6,1006,2016-11-02,5392,597,36,2036.79,n,error,44
8,1008,2016-07-02,5430,212,21,3901.22,n,error,26


Fill Missing Price values with Mean Price

In [52]:
df_dropped['price'].fillna(value=np.round(df.price.mean(),decimals=2),
                                inplace=True)

Fill Missing user_type values with value from previous row (forward fill) 

In [53]:
df_dropped['user_type'].fillna(method='ffill',inplace=True)

Fill Missing user_type values with value from next row (backward fill)

In [54]:
df_dropped['user_type'].fillna(method='bfill',inplace=True)

### Duplicates

Drop Duplicate serial_no rows

In [56]:
# sample duplicates
display(df_dropped[df_dropped.duplicated(subset=['serial_no'])].head())
print("Shape of df={}".format(df_dropped.shape))

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week
64,-1,2016-01-24,5945,103,24,586.75,d,loyal_existing,3
103,-1,2016-03-02,5821,202,11,2769.42,a,new,9
162,-1,2016-01-15,5792,864,34,3549.00,a,new,2
173,-1,2016-01-02,5040,291,3,3965.65,a,new,53
200,-1,2016-01-17,5192,1072,2,799.74,b,new,2


Shape of df=(969, 9)


In [57]:
df_dropped.drop_duplicates(subset=['serial_no'],inplace=True)

In [59]:
# updated dataframe
display(df_dropped.head())
print("Shape of df={}".format(df_dropped.shape))

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week
2,1002,2016-06-01,5439,488,17,2817.62,n,error,22
4,1004,2016-01-25,5001,580,30,2174.16,n,error,4
5,1005,2016-01-25,5405,601,31,1571.84,n,error,4
6,1006,2016-11-02,5392,597,36,2036.79,n,error,44
8,1008,2016-07-02,5430,212,21,3901.22,n,error,26


Shape of df=(939, 9)


Remove rows which have less than 3 attributes with non-missing data

In [62]:
display(df.dropna(thresh=3).head())
print("Shape of df={}".format(df.dropna(thresh=3).shape))

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week
0,1000,NaT,-101,0,28,NaN,n,error,0
1,1001,NaT,5395,411,25,4177.95,n,error,0
2,1002,2016-06-01,5439,488,17,2817.62,n,error,22
3,1003,NaT,5821,202,35,8576.58,n,error,0
4,1004,2016-01-25,5001,580,30,2174.16,n,error,4


Shape of df=(1000, 9)


### Encode Categoricals

One Hot Encoding using get_dummies()

In [63]:
display(pd.get_dummies(df,columns=['user_type']).head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_class,purchase_week,user_type_a,user_type_b,user_type_c,user_type_d,user_type_n
0,1000,NaT,-101,0,28,NaN,error,0,0,0,0,0,1
1,1001,NaT,5395,411,25,4177.95,error,0,0,0,0,0,1
2,1002,2016-06-01,5439,488,17,2817.62,error,22,0,0,0,0,1
3,1003,NaT,5821,202,35,8576.58,error,0,0,0,0,0,1
4,1004,2016-01-25,5001,580,30,2174.16,error,4,0,0,0,0,1


Label Mapping

In [66]:
type_map={'a':0,'b':1,'c':2,'d':3,np.NAN:-1}
df['encoded_user_type'] = df.user_type.map(type_map)
display((df.tail()))

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week,encoded_user_type
995,1995,2016-06-02,5229,378,38,2962.47,c,existing,22,2.0
996,1996,2016-01-13,5639,1002,26,4674.63,b,new,2,1.0
997,1997,2016-10-01,5787,180,25,4060.43,a,new,39,0.0
998,1998,2016-10-02,5463,868,8,3867.00,b,new,39,1.0
999,1999,2016-06-01,5283,918,27,2370.00,b,new,22,1.0


### Random Sampling Data From Data Frame

In [67]:
display(df.sample(frac=0.2, replace=True, random_state=42).head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week,encoded_user_type
102,1102,2016-01-27,5439,488,41,3158.98,b,new,4,1.0
435,1435,2016-07-02,5596,392,24,303.93,b,new,26,1.0
860,1860,2016-01-14,5916,314,8,1184.57,d,loyal_existing,2,3.0
270,1270,2016-01-17,5185,1040,27,1414.92,d,loyal_existing,2,3.0
106,1106,2016-01-27,5392,597,4,3014.35,a,new,4,0.0


### Normalizing Numeric Values

Normalize price values using  **Min-Max Scaler**

In [71]:
df_normalized = df.dropna().copy()
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(df_normalized['price'].values.reshape(-1,1))
df_normalized['price'] = np_scaled.reshape(-1,1)

In [72]:
display(df_normalized.head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week,encoded_user_type
32,1032,2016-02-01,5475,110,28,0.440488,b,new,5,1.0
34,1034,2016-01-24,5542,265,4,0.334589,b,new,3,1.0
35,1035,2016-06-01,5596,392,39,0.187270,d,loyal_existing,22,3.0
39,1039,2016-05-02,5520,322,6,0.218581,c,existing,18,2.0
40,1040,2016-05-01,5834,729,12,0.368064,a,new,17,0.0


Normalize quantity purchased values using  **Robust Scaler**

In [76]:
df_normalized = df.dropna().copy()
robust_scaler = preprocessing.RobustScaler()
rs_scaled = robust_scaler.fit_transform(df_normalized['quantity_purchased'].values.reshape(-1,1))
df_normalized['quantity_purchased'] = rs_scaled.reshape(-1,1)

In [77]:
display(df_normalized.head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week,encoded_user_type
32,1032,2016-02-01,5475,110,0.30,3952.14,b,new,5,1.0
34,1034,2016-01-24,5542,265,-0.90,3004.15,b,new,3,1.0
35,1035,2016-06-01,5596,392,0.85,1685.37,d,loyal_existing,22,3.0
39,1039,2016-05-02,5520,322,-0.80,1965.66,c,existing,18,2.0
40,1040,2016-05-01,5834,729,-0.50,3303.81,a,new,17,0.0


### Data Summarization

Condition Based Categorization

In [78]:
print("Mean price of items purchased by user_type=a :: {}".format(df['price'][df['user_type']=='a'].mean()))

Mean price of items purchased by user_type=a :: 2324.306528301886


Condition Based Counts

In [79]:
print(df['purchase_week'].value_counts())

4     164
3     150
2     127
44     57
39     50
9      50
5      49
13     48
31     46
53     41
35     41
26     39
22     37
0      31
18     30
17     20
48     20
Name: purchase_week, dtype: int64


### Grouping

Group By certain attributes

In [80]:
print(df.groupby(['user_class'])['quantity_purchased'].sum())

user_class
error               641
existing           4785
loyal_existing     4991
new               10722
Name: quantity_purchased, dtype: int32


Group By with different aggregate functions

In [82]:
display(df.groupby(['user_class'])['quantity_purchased'].agg([np.sum,
                                                                np.mean,
                                                                np.count_nonzero]))

,sum,mean,count_nonzero
user_class,,,
error,641,20.677419,31
existing,4785,21.361607,224
loyal_existing,4991,21.794760,229
new,10722,20.779070,516


Group by specific aggregate functions for each attribute

In [83]:
display(df.groupby(['user_class','user_type']).agg({'price':np.mean,
                                                        'quantity_purchased':np.max}))

price  quantity_purchased
user_class     user_type                                 
error          n          2591.445517                  39
existing       c          2177.428356                  41
loyal_existing d          2538.310320                  41
new            a          2324.306528                  41
               b          2281.704538                  41

Group by with multiple agg for each attribute

In [85]:
display(df.groupby(['user_class','user_type']).agg({'price':{
                                                                'total_price':np.sum,
                                                                'mean_price':np.mean,
                                                                'variance_price':np.std,
                                                                'count':np.count_nonzero},
                                                   'quantity_purchased':np.sum})) 

C:\Users\Asghar\Anaconda3\lib\site-packages\pandas\core\groupby\generic.py:1315: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


price                                     \
                         total_price   mean_price variance_price  count   
user_class     user_type                                                  
error          n            75151.92  2591.445517    2060.904906   31.0   
existing       c           476856.81  2177.428356    1550.124582  224.0   
loyal_existing d           555889.96  2538.310320    1654.145173  229.0   
new            a           615941.23  2324.306528    1546.284646  271.0   
               b           543045.68  2281.704538    1632.730199  245.0   

                         quantity_purchased  
                                        sum  
user_class     user_type                     
error          n                        641  
existing       c                       4785  
loyal_existing d                       4991  
new            a                       5305  
               b                       5417

### Pivot Tables

In [86]:
display(df.pivot_table(index='date', columns='user_type', 
                         values='price',aggfunc=np.mean))

user_type,a,b,c,d,n
date,,,,,
2016-01-01,4584.326667,1038.790000,1570.703333,1585.100000,4789.750000
2016-01-02,3505.659000,1366.090000,2251.804444,3691.450000,NaN
2016-01-13,2024.256667,1959.332857,4485.370000,2503.468750,924.490000
2016-01-14,1929.568000,1733.330000,2034.792222,2457.934000,NaN
2016-01-15,2082.110000,2606.407500,2487.152500,2392.768333,NaN
2016-01-16,2017.843636,1104.698333,1484.643333,3572.522500,NaN
2016-01-17,1623.555000,998.740000,1879.910000,2634.440909,NaN
2016-01-18,1754.754286,3117.086000,2383.663333,2229.773333,2889.640000
2016-01-19,3251.982500,2558.268333,1968.831667,1075.875000,NaN


### Stack a Dataframe

In [87]:
print(df.stack())

0    serial_no                            1000
     user_id                              -101
     product_id                              0
     quantity_purchased                     28
     user_type                               n
     user_class                          error
     purchase_week                           0
1    serial_no                            1001
     user_id                              5395
     product_id                            411
     quantity_purchased                     25
     price                             4177.95
     user_type                               n
     user_class                          error
     purchase_week                           0
2    serial_no                            1002
     date                  2016-06-01 00:00:00
     user_id                              5439
     product_id                            488
     quantity_purchased                     17
     price                             2817.62
     user_typ